In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('train.csv', sep='\t')#, index_col='id')

In [ ]:
df_train.info()

In [ ]:
from tqdm import tqdm

def ClearTag(TextData): #очиста от тэгов в столбце
    texts_without_tags = []
    
    for t in tqdm(TextData):
        texts_without_tags.append(BeautifulSoup(t, 'lxml').text)
    
    return texts_without_tags

In [ ]:
df_train['name'] = ClearTag(df_train['name'])

In [ ]:
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
#import math # Корень квадратный.

morph=pymorphy2.MorphAnalyzer()

In [ ]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁёa-zA-Z]+\-[А-Яа-яЁёa-zA-Z]+|[А-Яа-яЁёa-zA-Z]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
                
    tokens_eng=re.findall('[a-zA-Z]+\-[a-zA-Z]+|[a-zA-Z]+', text)
    for t in tokens_eng:
        words.append(t)
    
    return ' '.join(words)

In [ ]:
getMeaningfullWords("Пиздец Account подкрался manger не заметно. Красивая ты вошла в мою грешную жизнь")

In [ ]:
num = df_train['name'].size #считываем размер выборки
#num = 10

clean_train_words = [] #собираем "мешок слов"

for i in tqdm(range(0, num)):
    clean_train_words.append(getMeaningfullWords(df_train['name'][i]))

In [ ]:
#сохранение резельтатов вычислений
# !!!ЗАПУСКАТЬ ОТДЕЛЬНО!!!
# ПРИ НЕОБХОДИМОСТИ СОХРАНИТЬ ДАННЫЕ В ФАЙЛ

filename = 'clean_train_words.csv'
file_save = open(filename, "w")

for item in clean_train_words:
    file_save.write(item+'\n')

file_save.close()

In [3]:
#LOAD clean_train_words from the file
filename = 'clean_train_words.csv'
file_load = open(filename)

text = file_load.read()
clean_train_words = text.split('\n')[1:]

file_load.close()

In [8]:
#SAVE train_data_features into the file
import _pickle as cPickle

filename = 'train_data_features.csv'

with open('train_data_features.csv', 'wb') as f:
    cPickle.dump(train_data_features, f)

MemoryError: 

In [ ]:
#LOAD train_data_features from the file
import _pickle as cPickle

filename = 'train_data_features.csv'

with open(filename, 'wb') as f:
    train_data_features = cPickle.load(f)

In [ ]:
df_train['name'][12]

In [ ]:
#clean_train_words

In [4]:
from sklearn.feature_extraction.text import CountVectorizer 

In [5]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 

In [11]:
%%time
train_data_features = vectorizer.fit_transform(clean_train_words)

Wall time: 2.63 s


In [7]:
%%time
#train_data_features = train_data_features.toarray()

Wall time: 6.91 s


In [12]:
train_data_features.shape

(200000, 5000)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train_data_features, df_train['target'], test_size=0.33, random_state=123)

In [15]:
print('X_train ', X_train.shape)
print('X_test ',X_test.shape)
print('y_train ', y_train.shape)
print('y_test ',y_test.shape)

X_train  (134000, 5000)
X_test  (66000, 5000)
y_train  (134000,)
y_test  (66000,)


In [16]:
%%time
from sklearn.ensemble import RandomForestClassifier
model_RFC = RandomForestClassifier(n_estimators = 100, n_jobs = -1) 

Wall time: 92.3 ms


In [17]:
%%time
model_RFC.fit( X_train, y_train )

Wall time: 11min 14s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
predict = model_RFC.predict(X_test)

In [20]:
print(classification_report(y_test, predict))

             precision    recall  f1-score   support

          0       0.54      0.68      0.60     35162
          1       0.47      0.33      0.39     30838

avg / total       0.51      0.51      0.50     66000



In [21]:
roc_auc_score(y_test, predict)

0.5033801272170464

 <b color='red'>НЕЗАПУСКАТЬ!!!</b>

In [ ]:
df_train = pd.read_csv('train.csv', sep='\t', index_col='id')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('target', axis=1), df_train['target'], 
                                                    test_size=0.30, random_state=123)

X_test['target'] = y_test

In [ ]:
X_test.to_csv('train1900.csv', sep='\t', index_label='id')

In [18]:
import _pickle as cPickle
with open('RFC.pcl', 'wb') as f:
    cPickle.dump(model_RFC, f)


In [ ]:
import _pickle as cPickle
with open('RFC.pcl', 'rb') as f:
    model_RFC = cPickle.load(f)